# Free energy calculations from $\lambda$-metadynamics using bootstrapping

In this notebook, we will break down `bootstrap_estimator.py` to show how we use bootstrapping to calculate the free energy differences and their uncertainties from $\lambda$-metadynamics. We will not go over all the functions but just the most important ones for helping the user understand the details of `bootstrap_estimator.py`. Note that 

In [146]:
import os
import sys
import time
import sparse
import plumed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from scipy.sparse import csr_matrix
from prettytable import PrettyTable

In [174]:
def block_bootstrap(traj, n_blocks, dt, file_path, B, T=298.15, truncate=0, CV='lambda'):
    """
    Calculates the free energy difference and its uncertainty from an
    alchemical metadynamics using block bootstrp.

    Parameters
    ----------
    traj       (PlumedDataFrame): The trajectory data read from a COLVAR file.
    n_blocks   (int): The number of blocks.
    dt         (float): STRIDE in ps
    file_path  (str): The path of the output file fes*dat
    B          (int): The number of bootstrap iterations
    T          (float): The simulation temperature.
    truncate    (float): The fraction of the time series to be truncated.

    Returns
    -------
    df         (float): The free energy difference between the coupled and uncoupled state.
    df_err     (float): The uncertainty of the free energy difference
    """
    k = 1.38064852E-23   # Boltzmann constant
    N_a = 6.02214076E23  # Avogadro's number
    kT = k * T * N_a / 1000  # kT in kJ/mol

    n = int(len(traj) * (1 - truncate))   # number of data points considered
    # make sure the number of frames is a multiple of nblocks (discard the first few frames)
    n = (n // n_blocks) * n_blocks
    b_size = n / np.array(n_blocks) * dt  # units: ps
    bias = np.array(traj["metad.bias"])
    bias -= np.max(bias)  # avoid overflows
    # shape: (nblocks, nframes in one block), weight for each point
    w = np.exp(bias / kT)[-n:].reshape((n_blocks, -1))

    fes_file = open(file_path, 'w')
    fes_file.write(f'{CV}    f             f_err\n')
    isState, pop, fes, fes_err = [], [], [], []
    for i in range(int(np.max(traj[f"{CV}"])) + 1):
        # 1 if in state i
        isState.append(np.int_(traj[f"{CV}"] == i)
                       [-n:].reshape((n_blocks, -1)))
        # draw samples from np.arange(n_blocks), size refers the output size
        boot = np.random.choice(n_blocks, size=(B, n_blocks))
        # isState[boot]: 3D array, shape of pop: (B,)
        pop.append(np.average(isState[i][boot].astype('float16'), axis=(1, 2), weights=w[boot]))
        #A = np.average(isState[i][boot].astype('float16') * w[boot].astype('float16'))
        #pop.append(np.average(A, axis=(1,2)))
        #pop.append((coo_matrix(isState[i][boot]).mutiply(coo_matrix(w[boot].astype('float16')))).mean(axis=(1, 2)))
        #print(w[boot].shape)
        #print(isState[i][boot].astype(bool))
        #print(w[boot][isState[i][boot].astype(bool)].shape)
        print(w.shape)
        print(boot.shape)
        print(w[boot].shape)
        print(isState[i][boot].nbytes)
        print(isState[i][boot].astype(bool).nbytes)
        print(sparse.COO(isState[i][boot]))
        print(sparse.COO(isState[i][boot]).size)
        print(sparse.COO(isState[i][boot].astype(bool)))
        print(sparse.COO(isState[i][boot].astype(bool)).size)
        #print(np.ma.array(w[boot], mask=[isState[i][boot].astype(bool)]).shape)
        #print(np.ma.array(w[boot], mask=[isState[i][boot].astype(bool)]))
        pop.append(np.average(isState[i][boot].astype(bool), axis=(1, 2), weights=w[boot]))
        fes_boot = np.log(pop[i])
        fes.append(np.log(np.average(isState[i], weights=w)))
        if np.sum(np.isinf(fes_boot)) > 0:   # in case that there are zeros in pop[i]
            warn_str = f'{np.sum(np.isinf(fes_boot))} out of {B} bootstrap iterations had 0 probability.'
            print('=' * int((len(warn_str) - 9 ) / 2) + ' Warning ' + '=' * int((len(warn_str) - 9) / 2))
            print(warn_str)
            print('=' * len(warn_str))
            fes_boot[np.isinf(fes_boot)] = float('nan')  # replace inf or -inf with nan
        fes_err.append(np.nanstd(fes_boot))   # ignore nan when calculating std

        # Write fes*dat
        fes_file.write(f'  {i}      {fes[-1]: .6f}    {fes_err[-1]: .6f}\n')

    # Free energy difference (kT) between the coupled and uncoupled states
    df_boot = np.log(pop[0]/pop[-1])
    popA0 = np.average(isState[0], weights=w)  # coupled state
    popB0 = np.average(isState[-1], weights=w)
    df = np.log(popA0 / popB0)
    if np.sum(np.isinf(df_boot)) > 0:   # in case that there are zeros in popA0/popB0
        df_boot[np.isinf(df_boot)] = float('nan')  # replace inf or -inf with nan
    df_err = np.nanstd(df_boot)    # ignore nan when calculating std

    return df, df_err

In [175]:
def average_bias(hills, avg_frac):
    """
    Averages the bias over a certain last portion of the simulation. 

    Parameters
    ----------
    hills     (PlumeDataFrame): The data read from HILLS. 
    avg_frac  (int): The fraction of the simulation that the data will be averaged when reweighting.

    Returns
    ----------
    hills_avg (PlumedDataFrame): The time-averaged bias.
    """
    n0 = int(len(hills) * (1 - avg_frac))   # number of data points considered
    # the weights for the first n0 points are 1
    w = np.hstack((np.ones(n0), np.linspace(1, 0, len(hills) - n0)))
    hills_avg = hills.copy()
    hills_avg.height *= w
    return hills_avg

## Step 1: Setting up input parameters and clean up the directory

In [176]:
%%bash
#rm COLVAR_SUM_BIAS
#rm bck.0.COLVAR_SUM_BIAS
#rm -r FES

In [177]:
np.random.seed(1994)
folder ='./'
p_input = folder + 'plumed_sum_bias.dat'
colvar = folder + 'COLVAR_SUM_BIAS'
hills = folder + 'HILLS_2D'
n_blocks = [20]
B = 200   # number of bootstrap iterations
T = 298.15  
truncate = 0.5
avg_frac = 0.2
factor_s = 0.02

In [178]:
def read_plumed_output(plumed_output):
    """
    Parameters
    ----------
    plumed_output (str): The filename of the plumed output (such as HILLS or COLVAR files) to be read
    """
    data_original = plumed.read_as_pandas(plumed_output)
    data = data_original[~data_original["time"].duplicated(keep='last')]  # deduplicate time frames
    data = data.dropna()        # drop N/A in case that there is any
    data = data.reset_index()   # reset the index of the data frame, after this an column "index" will be added
    data = data.drop(columns=["index"])    # drop the index column
    if len(data) == len(data_original):
        pass    # the plumed output file won't be modified and there will be no backup
    else:
        # the data in the plumed output file will be replaced with the deduplicated time series
        backup = plumed_output + '_backup'
        os.system(f'mv {plumed_output} {backup}')
        plumed.write_pandas(data, plumed_output)  
    return data

## Step 2: Run the plumed driver for reweighting

In [179]:
%%bash
cat lambda_MetaD_data/plumed_sum_bias.dat

theta: READ FILE=COLVAR VALUES=theta IGNORE_TIME IGNORE_FORCES
lambda: READ FILE=COLVAR VALUES=lambda IGNORE_TIME IGNORE_FORCES

METAD ...
ARG=theta,lambda 
SIGMA=0.5,0.0001     # small SIGMA ensure that the Gaussian approaximate a delta function
HEIGHT=0
PACE=5000000        # should be nstexpanded
GRID_MIN=-pi,0   # index of alchemical states starts from 0
GRID_MAX=pi,8    # we have 6 states in total
GRID_BIN=100,8
TEMP=298
BIASFACTOR=10
LABEL=metad    
FILE=HILLS_2D_modified
RESTART=YES
... METAD

PRINT STRIDE=1 ARG=theta,lambda,metad.bias FILE=COLVAR_SUM_BIAS


In [180]:
# hills_data = read_plumed_output('HILLS_2D')
colvar_data = read_plumed_output(folder + 'COLVAR')
hills_avg = average_bias(plumed.read_as_pandas(hills), avg_frac)
plumed.write_pandas(hills_avg, hills + "_modified")

<ipython-input-178-3d3c39aa6908>:7: UserWarning: cannot load PLUMED instance, conversions will not be available
  data_original = plumed.read_as_pandas(plumed_output)
<ipython-input-180-82aafd29ff69>:3: UserWarning: cannot load PLUMED instance, conversions will not be available
  hills_avg = average_bias(plumed.read_as_pandas(hills), avg_frac)


In [181]:
%%bash
source /usr/local/gromacs/bin/GMXRC
source /Users/Wei-TseHsu/Documents/Software/PLUMED/plumed2/sourceme.sh
plumed driver --plumed plumed_sum_bias.dat --noatoms

PLUMED: PLUMED is starting
PLUMED: Version: 2.8.0-dev (git: 63008b018) compiled on Nov 21 2020 at 02:44:56
PLUMED: Please cite these papers when using PLUMED [1][2]
PLUMED: For further information see the PLUMED web page at http://www.plumed.org
PLUMED: Root: /Users/Wei-TseHsu/Documents/Software/PLUMED/plumed2/
PLUMED: For installed feature, see /Users/Wei-TseHsu/Documents/Software/PLUMED/plumed2//src/config/config.txt
PLUMED: Molecular dynamics engine: driver
PLUMED: Precision of reals: 8
PLUMED: Running over 1 node
PLUMED: Number of threads: 1
PLUMED: Cache line size: 512
PLUMED: Number of atoms: 0
PLUMED: File suffix: 
PLUMED: FILE: plumed_sum_bias.dat
PLUMED: Action READ
PLUMED:   with label theta
PLUMED:   with stride 1
PLUMED:   reading data from file COLVAR
PLUMED:   reading value theta and storing as theta
PLUMED: Action READ
PLUMED:   with label lambda
PLUMED:   with stride 1
PLUMED:   reading data from file COLVAR
PLUMED:   reading value lambda and storing as lambda
PLUMED: A

## Step 3: Calculate the free energy surface and its uncertainty

In [182]:
if len(n_blocks) == 1:
    multi_b_size = False
    n_blocks = n_blocks
else:
    multi_b_size = True
traj = plumed.read_as_pandas(colvar)
df_results = []
n = int(len(traj) * (1 - truncate))  # number of data points considered
n = (n // np.array(n_blocks)) * n_blocks
b_sizes = n / np.array(n_blocks) * factor_s  # units: ps
for i in range(len(n_blocks)):
    if multi_b_size is False:  # only one fes_*.dat
        fes_path = f'{folder}/fes_truncate_{truncate}_bsize_{b_sizes[i]}_avg_{avg_frac}.dat'
    else:
        fes_path = f'{folder}FES/fes_bsize_{b_sizes[i]}.dat'
    df_results.append(block_bootstrap(
        traj, n_blocks[i], factor_s, fes_path, B, T, truncate, 'lambda'))

<ipython-input-182-a8bb8a83d1e7>:6: UserWarning: cannot load PLUMED instance, conversions will not be available
  traj = plumed.read_as_pandas(colvar)


(20, 25000)
(200, 20)
(200, 20, 25000)
800000000
100000000
<COO: shape=(200, 20, 25000), dtype=int64, nnz=37061204, fill_value=0>
100000000
<COO: shape=(200, 20, 25000), dtype=bool, nnz=37061204, fill_value=False>
100000000
(20, 25000)
(200, 20)
(200, 20, 25000)
800000000
100000000
<COO: shape=(200, 20, 25000), dtype=int64, nnz=16927112, fill_value=0>
100000000
<COO: shape=(200, 20, 25000), dtype=bool, nnz=16927112, fill_value=False>
100000000
(20, 25000)
(200, 20)
(200, 20, 25000)
800000000
100000000
<COO: shape=(200, 20, 25000), dtype=int64, nnz=9218973, fill_value=0>
100000000
<COO: shape=(200, 20, 25000), dtype=bool, nnz=9218973, fill_value=False>
100000000
(20, 25000)
(200, 20)
(200, 20, 25000)
800000000
100000000
<COO: shape=(200, 20, 25000), dtype=int64, nnz=6953251, fill_value=0>
100000000
<COO: shape=(200, 20, 25000), dtype=bool, nnz=6953251, fill_value=False>
100000000
(20, 25000)
(200, 20)
(200, 20, 25000)
800000000
100000000
<COO: shape=(200, 20, 25000), dtype=int64, nnz=54

KeyboardInterrupt: 

In [59]:
n = int(len(traj) * (1 - truncate))   # number of data points considered
# make sure the number of frames is a multiple of nblocks (discard the first few frames)
n = (n // n_blocks) * n_blocks
b_size = n / np.array(n_blocks) * dt  # units: ps
bias = np.array(traj["metad.bias"])
bias -= np.max(bias)  # avoid overflows
# shape: (nblocks, nframes in one block), weight for each point
w = np.exp(bias / kT)[-n:].reshape((n_blocks, -1))

TypeError: unsupported operand type(s) for //: 'int' and 'list'

In [ ]:
dt = 0.02
k = 1.38064852E-23   # Boltzmann constant
N_a = 6.02214076E23  # Avogadro's number
kT = k * T * N_a / 1000  # kT in kJ/mol
traj = plumed.read_as_pandas(colvar)
n = int(len(traj) * (1 - truncate))   # number of data points considered
# make sure the number of frames is a multiple of nblocks (discard the first few frames)
n = (n // n_blocks[0]) * n_blocks[0]
b_size = n / np.array(n_blocks) * dt  # units: ps
bias = np.array(traj["metad.bias"])
w = np.exp(bias / kT)[-n:].reshape((n_blocks[0], -1))

In [46]:
w[boot]

NameError: name 'boot' is not defined

In [24]:
traj

,time,theta,lambda,metad.bias
0,0.0,-3.006085,0.0,113.205556
1,1.0,3.094475,0.0,113.762690
2,2.0,-2.923163,0.0,112.806400
3,3.0,-2.995856,0.0,113.159989
4,4.0,-3.018957,0.0,113.260142
...,...,...,...,...
999996,999996.0,-2.603765,7.0,72.497909
999997,999997.0,-2.341541,7.0,70.989072
999998,999998.0,-2.329528,7.0,70.913153
999999,999999.0,-2.296362,8.0,76.696924


In [44]:
A = np.array([1,0,1,1,0,0,0])
np.average(A.astype(bool), weights=np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]).astype('float16'))

0.42857142857142855

In [27]:
A = np.array(A, dtype=bool)
A.nbytes

700000

In [173]:
A = sparse.COO(np.array([1, 2, 3, 4, 5]))
B = sparse.COO(np.array([0, 1, 0, 1, 1]))
np.ma.array(A.todense(), mask=B.todense())

masked_array(data=[1, --, 3, --, --],
             mask=[False,  True, False,  True,  True],
       fill_value=999999)

In [33]:
B = np.array([1,2,3,4])
B = np.array(B, dtype=bool)
B

array([ True,  True,  True,  True])

In [30]:
3/7

0.42857142857142855

In [ ]:
import scipy
A = scipy.sparse.coo_matrix(np.random.rand(3, 4))
A.toarray()

In [ ]:
A = [1,2,3,4,5]
B=[1,2,3,4,5]
coo_matrix(A).multiply(coo_matrix(B)).mean()

In [ ]:
np.average(A, weights=B)

In [ ]:
(coo_matrix(A).multiply(coo_matrix(B))/(coo_matrix(B).sum())).sum()

In [ ]:
sparse.COO(np.array(A))

In [124]:
A = np.array([1,2,3,4,5]*100000)
#B = np.array([0.1, 0.1, 0.1, 0.2, 0.2])
#np.mean(np.multiply(A, B))

In [ ]:
np.average(A, weights=B)

In [ ]:
np.sum(np.multiply(A, B))/np.sum(B)

In [125]:
coo_matrix(A*5).size

500000

In [169]:
np.sum(sparse.COO(A*5))

7500000

In [159]:
A.nbytes

4000000

In [214]:
A = np.array([[2, 0], [1, 1]])
B = np.array([[1, 0], [0, 0], [0, 0]])
A[B]

array([[[1, 1],
        [2, 0]],

       [[2, 0],
        [2, 0]],

       [[2, 0],
        [2, 0]]])

In [189]:
(20, 25000)
(200, 20)
(200, 20, 25000)

(200, 20, 25000)

In [211]:
w.shape
boot = np.random.choice(20, size=(200, 20))
boot

array([[ 3, 15,  1, ..., 11,  8, 10],
       [14,  2,  6, ...,  0, 14,  3],
       [19,  5,  2, ...,  1,  5,  4],
       ...,
       [11,  3,  1, ...,  8,  6,  8],
       [ 5,  0,  9, ...,  8,  9, 15],
       [17,  8,  9, ...,  2, 16, 10]])

In [208]:
boot.shape

(200, 20)

In [209]:
w[boot]

IndexError: index 25 is out of bounds for axis 0 with size 20

In [206]:
w[boot].shape

(200, 20, 25000)